In [1]:
import pandas as pd

inference_result_df = pd.read_csv('inference_result_RNN_vanilla_2.csv', usecols=['Predicted_label', 'ReviewerID'])
inference_result_df.head()

,review_id,user_id,business_id,stars,date,useful,text,business_name,user_name,timestamp,gender
0,Pm6qydbW58BB0zdWCPlBmg,Skzdl0sWhW88525a1vr59g,ujmEBvifdJM6h6RLv4wQIg,1.0,2018-05-08 19:27:59,3.0,"Went in for a broken finger, was asked if I wa...",MountainView Hospital,Renee,1.525800e+09,female
1,US_dupN8RTXGNdmDeksQIg,Skzdl0sWhW88525a1vr59g,I5TnTKHzJuLA0YLZPGCNwQ,5.0,2016-09-03 02:52:45,0.0,Absolutely love this place! Great food for the...,Teriyaki Boy Healthy Grill,Renee,1.472864e+09,female
2,b5mIIyHZKwYuaFQPnO1BCw,Skzdl0sWhW88525a1vr59g,3dC3opMY67zrquz_yYc-tw,1.0,2017-04-25 21:38:51,0.0,"Frequent this place a lot, well today there wa...",Pepe's Tacos,Renee,1.493149e+09,female
3,afOVxx5-s_ZQyhkhk2drNA,Skzdl0sWhW88525a1vr59g,_GqNHoWtBOksNcfVjnl0YQ,5.0,2017-04-11 20:54:24,0.0,"Phil Brown is a very knowledgeable person, he ...",Brown Law Offices,Renee,1.491937e+09,female
4,X3qb2HL3o-FMTJywO2THSA,Skzdl0sWhW88525a1vr59g,WSGHEQdcdbBWXDpna99EiQ,5.0,2016-12-04 03:10:34,0.0,I absolutely love this place. It's delicious a...,Teriyaki Madness,Renee,1.480817e+09,female


In [3]:

inference_result_grouped_df = inference_result_df.groupby('ReviewerID').agg({'Predicted_label': 'value_counts'})
inference_result_grouped_df.head()

Predicted_label
ReviewerID             Predicted_label                 
---1lKK3aKOuomHnwAkAow 1                             97
                       0                             31
---PLwSf5gKdIoVnyRHgBA 1                              2
--0RtXvcOIE4XbErYca6Rw 0                              1
--0VhLFv2XfEKHL0YSCSuw 1                              1

In [4]:
majority_voting_result_ser = inference_result_df.groupby('ReviewerID').max()
majority_voting_result_ser.head()

,Predicted_label
ReviewerID,
---1lKK3aKOuomHnwAkAow,1
---PLwSf5gKdIoVnyRHgBA,1
--0RtXvcOIE4XbErYca6Rw,0
--0VhLFv2XfEKHL0YSCSuw,1
--2DMAd_kGHgbF88H2SLuw,1


In [5]:
majority_voting_result_df = pd.DataFrame(majority_voting_result_ser.Predicted_label).reset_index()


In [6]:
majority_voting_result_df.columns = ['user_id', 'predicted_gender']
majority_voting_result_df.head()

,user_id,predicted_gender
0,---1lKK3aKOuomHnwAkAow,1
1,---PLwSf5gKdIoVnyRHgBA,1
2,--0RtXvcOIE4XbErYca6Rw,0
3,--0VhLFv2XfEKHL0YSCSuw,1
4,--2DMAd_kGHgbF88H2SLuw,1


In [42]:
majority_voting_result_df.predicted_gender.value_counts()

NameError: name 'majority_voting_result_df' is not defined

In [1]:
import pandas as pd


disclosed_flag = True
if disclosed_flag:
    disclosed_dataset_df = pd.read_csv('disclosed_dataset.csv')
    mapping = {'female' : 1, 'male' : 0}
    disclosed_dataset_df.replace({'gender': mapping}, inplace=True)
    test_df = disclosed_dataset_df.head()
else:
    undisclosed_csv_df = pd.read_csv('undisclosed_dataset.csv')
    undisclosed_csv_df.head()

In [28]:
test_df = disclosed_dataset_df.iloc[1:100]

In [94]:
test_tmp = test_df.groupby('user_id').apply(lambda g: g.gender.mode())
test_tmp.head()

,0
user_id,
3Y25VDfnQVcuc33T-U3Z6A,1
CHuPGiofxPPIxK5cXAS4qg,1
HjyfcIOv1PoLJAoqt-J4GQ,1
MJYkCvxASq9GKEXDrmhB0Q,1
PgjzouAp8FyOwh6J83D4rA,1


In [57]:
ser = test_tmp.gender.groupby('user_id').max(level=0)
ser

user_id
3Y25VDfnQVcuc33T-U3Z6A     4
CHuPGiofxPPIxK5cXAS4qg    16
HjyfcIOv1PoLJAoqt-J4GQ     8
MJYkCvxASq9GKEXDrmhB0Q     3
PgjzouAp8FyOwh6J83D4rA     3
Skzdl0sWhW88525a1vr59g     4
i5aJIiQk91PK9Z0aYJpOrg    61
Name: gender, dtype: int64

In [20]:
merged_df = pd.merge(majority_voting_result_df, undisclosed_csv_df, on='user_id', how='outer')


In [21]:
# majority_voting_result_df.count
undisclosed_csv_df.user_id.value_counts()

bLbSNkLggFnqwNNzzq-Ijw    2354
PKEzKWv_FktMm2mGPjwd0Q    1822
QJI9OSEn6ujRCtrX06vs1w    1496
YRcaNlwQ6XXPFDXWtuMGdA    1234
dIIKEfOgo0KqUfGQvGikPg    1198
                          ... 
_dz_a3GSGZ0unVKbRFC32A       1
FlcqUPN8UArhD2J_YlurWA       1
9IvdoNXPoUGCHZG9OZ2LEQ       1
w9nQhf2ppUqhuDxmumkkdw       1
CqS4mNQkd_FYgtjNxgLvew       1
Name: user_id, Length: 597765, dtype: int64

In [22]:
undisclosed_csv_df.count()

review_id        2416933
user_id          2416933
business_id      2416933
stars            2416933
date             2416933
useful           2416933
text             2416933
business_name    2416933
user_name        2416933
timestamp        2416933
gender           2416933
dtype: int64

In [23]:
majority_voting_result_df.count()

user_id             597749
predicted_gender    597749
dtype: int64

In [24]:
inference_result_grouped_df.count()

Predicted_label    771549
dtype: int64

In [25]:
merged_df.count()

user_id             2416933
predicted_gender    2416916
review_id           2416933
business_id         2416933
stars               2416933
date                2416933
useful              2416933
text                2416933
business_name       2416933
user_name           2416933
timestamp           2416933
gender              2416933
dtype: int64

In [26]:
merged_dropna_df = merged_df.dropna()
merged_dropna_df.count()

user_id             2416916
predicted_gender    2416916
review_id           2416916
business_id         2416916
stars               2416916
date                2416916
useful              2416916
text                2416916
business_name       2416916
user_name           2416916
timestamp           2416916
gender              2416916
dtype: int64

In [27]:
merged_dropna_df.head()

,user_id,predicted_gender,review_id,business_id,stars,date,useful,text,business_name,user_name,timestamp,gender
0,---1lKK3aKOuomHnwAkAow,1.0,7MAUYp00ICZUK14WBYlfxg,qmymSqVwHYRqdwfcBatzpQ,5.0,2010-11-23 18:10:24,4.0,So it is 5 stars when Josh(Im pretty sure it i...,Vintner Grill,Monera,1.290532e+09,unknown
1,---1lKK3aKOuomHnwAkAow,1.0,be61S6GrtjoINP4BW8xicg,jobP3ywRd3QNZ_GCoPG2DQ,3.0,2011-01-07 01:37:15,0.0,I came to see Tron in IMAX because it was cent...,Brenden Theatres,Monera,1.294361e+09,unknown
2,---1lKK3aKOuomHnwAkAow,1.0,5MKDmlI1_wJLKicPH4aqaw,mz9ltimeAIy2c2qf5ctljw,5.0,2011-05-01 15:46:15,1.0,Lets start out by saying that I am not a Beatl...,Cirque du Soleil - The Beatles LOVE,Monera,1.304258e+09,unknown
3,---1lKK3aKOuomHnwAkAow,1.0,KkNpgkf0iTUoSk5jB2LqdQ,HZdtHOEaKUL2SlWj5owgCA,5.0,2010-11-27 04:36:03,1.0,So if you hate pumping your own gas they have ...,Fabulous Freddy's Summerlin,Monera,1.290829e+09,unknown
4,---1lKK3aKOuomHnwAkAow,1.0,_U_ctMQkF3Smqo9mjpw1Iw,CWNMLT-ppaUjLMmrnYDPVg,5.0,2010-11-05 21:58:29,3.0,Why the hell does anyone go to Kona Grill? Un...,Roy's Restaurant,Monera,1.288991e+09,unknown


In [28]:
merged_dropna_df.to_csv('undisclosed_predicted_dataset.csv', index=False)

In [29]:
fields = ['business_id','predicted_gender','useful','text', 'stars', 'timestamp']
df_male = merged_dropna_df[fields]
df_male.head()

,business_id,predicted_gender,useful,text,stars,timestamp
0,qmymSqVwHYRqdwfcBatzpQ,1.0,4.0,So it is 5 stars when Josh(Im pretty sure it i...,5.0,1.290532e+09
1,jobP3ywRd3QNZ_GCoPG2DQ,1.0,0.0,I came to see Tron in IMAX because it was cent...,3.0,1.294361e+09
2,mz9ltimeAIy2c2qf5ctljw,1.0,1.0,Lets start out by saying that I am not a Beatl...,5.0,1.304258e+09
3,HZdtHOEaKUL2SlWj5owgCA,1.0,1.0,So if you hate pumping your own gas they have ...,5.0,1.290829e+09
4,CWNMLT-ppaUjLMmrnYDPVg,1.0,3.0,Why the hell does anyone go to Kona Grill? Un...,5.0,1.288991e+09
